In [ ]:
!pip install -q langchain-community cassio cassandra-driver langchain-openai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
from langchain_community.vectorstores.cassandra import Cassandra
from langchain_classic.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from datasets import load_dataset

import cassio

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.3 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

In [ ]:
ASTRA_DB_APPLICATION_TOKEN="AstraCS:JehISzchupzQyDIowQDgxeov:b36ef593dc4bc2da1c53b9f54e8172ea31e693bf0108117958c912a574775fa5"
ASTRA_DB_ID="cc23aaed-5738-427c-97fc-9a5e64e6f3fd"

OPENAI_API_KEY=""##Enter your open ai api key

In [ ]:
pdfreader=PdfReader('budget.pdf')

In [ ]:
from typing_extensions import Concatenate
raw_text=''
for i,page in enumerate(pdfreader.pages):
  content=page.extract_text()
  if content:
    raw_text+=content

In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [ ]:
llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY)
embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
astra_vector_store=Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None
)

In [ ]:
!pip install -U langchain

In [ ]:
from langchain_classic.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(chunk_size=800,
                                    chunk_overlap=200,
                                    length_function=len,
                                    separator='\n')
texts=text_splitter.split_text(raw_text)

In [ ]:
texts[:50]

['www.nbsm.com.npNEPAL BUDGET\n2082 -83 (2025 -26)\nHIGHLIGHTS FROM TAX PERSPECTIVE  TABLE OF CONTENTS\n15-27\n28-33\n34-44BUDGET OBJECTIVES\nBUDGET SUMMARY\nTAXATION GENERAL\nTAX AMNESTY SCHEME\nINCOME TAX\nVALUE ADDED TAX\nEXCISE DUTY03-04\n05-09\n10-11\n12-14\nCUSTOMS DUTY 45-64\n23FIVE OBJECTIVES OF BUDGET\n01 02 03 04 05\nAchieve High, \nsustainable and \nbroad -based \neconomic growth to \nreduce povertyFoster \nentrepreneurship \nand expand public \nand private \ninvestment to create \njobs.\nImprove the \nquality of public \nservice and \npromote good \ngovernanceEstablished social justice \nthrough social protection \nand development\nEnhance economic \nefficiency by \nincreasing the use \nof modern \ntechnology4PRIORITIES OF \nBUDGET 2082/83 \n(2025 -26)Entrepreneurship, employment, production and',
 'and development\nEnhance economic \nefficiency by \nincreasing the use \nof modern \ntechnology4PRIORITIES OF \nBUDGET 2082/83 \n(2025 -26)Entrepreneurship, employment, producti

In [ ]:
astra_vector_store.add_texts(texts[:50])
print("Inserted %i headlines"% len(texts[:50]))
astra_vector_index=VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines


In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): What is the total agriculture budget and the main focuses on agriculture?

QUESTION: "What is the total agriculture budget and the main focuses on agriculture?"


ANSWER: "The total agriculture budget is not directly specified in the provided context. However, the main focus on agriculture in the budget for the year 2082/83 (2025-26) includes policies to ensure cultivable land is not left unused and to establish land banks for utilizing farmable land. This is aimed at promoting more efficient use of agricultural resources."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9137] "and development
Enhance economic 
efficiency by 
increasing the use 
of modern 
tech ..."
    [0.9049] "digital  payments  more attractive  and affordable . 
•Startups  with turnover  up t ..."
    [0.9028] "Foreign Grant
Internal & Foreign 
Debt1315
5345
595SUMMARY OF BUDGET
Economic Growth ..."
    [0.8978] "-25.00%-20.00%-15.00%-10.00%-5.00%0.00%5.00%10.00%15.00%
2076/77 2077/78 2078/79 207 ..."

What's your next question (or type 'quit' to exit): quit
